In [4]:

import argparse
import cv2
import numpy as np
import abcclassify.abcclassify as abcc
import csv
import os.path
from sklearn import tree


In [5]:
# from 4822 to 16134 for P01
participant = abcc.videotracking(framerange=(4822, 16134))
participant.addtrackingdata("/home/zzalsdme/IDInteraction/paper/results/OpenFace/P01_front.openface")
groundtruth = abcc.loadExternalGroundTruth("/home/zzalsdme/IDInteraction/paper/results/Groundtruth/P01_attention.csv", participant)
# All the potential training frames
trainingframes = participant.frames
np.random.shuffle(trainingframes)



Loading tracking data from: /home/zzalsdme/IDInteraction/paper/results/OpenFace/P01_front.openface
OpenFace input detected


We want to explore different approaches to training the classifier.  We've already looked at sequential and random, and shown that random classification leads to much better performance.  

Rob suggested looking at areas that had poor performance, and focussing on these.  If we've got ground truth we *know* where has poor performance. Normally though we'll need to use cross validation to estimate the performance of the classifier


We want a window size such that the cross-validation scores are resonably stable.



In [6]:
maxsd = 0.05
batchsize = 50
participant.setClassificationMethod("random")
for batch in range(0,10):
    for f in range(0, batchsize):
        thisframe = trainingframes.pop()
        participant.setClassification(thisframe, \
                                      groundtruth.loc[thisframe]["state"], \
                                      testunset = True )
    vtc = abcc.videotrackingclassifier(participant)
    xvalscore = vtc.getCrossValidatedScore(cv=10)
    print str(xvalscore.mean()) + " " + str(xvalscore.std())
    print vtc.vto.numClassifiedFrames()
    
    
    



0.796666666667 0.163605215891
45
0.812777777778 0.124376221379
93
0.8 0.109730653541
140
0.85649122807 0.0727557770791
189
0.790365612648 0.124719069477
235
0.792266009852 0.113507104396
285
0.799715909091 0.105519572895
333
0.823301236459 0.100072954304
382
0.79959478506 0.126234836549
430
0.823581560284 0.111813766963
477
